In [1]:
from astropy.io import fits
import pandas as pd
import os

# Dizin içindeki tüm FITS dosyalarını al
fits_directory = '/Users/nurullah/Desktop/galaxy_abudance/sigma_karşılaştırma/'
fits_file_paths = [os.path.join(fits_directory, file) for file in sorted(os.listdir(fits_directory)) if file.endswith('.fits')]

# Excel dosyalarının kaydedileceği dizin
output_directory = '/Users/nurullah/Desktop/galaxy_abudance/sigma_karşılaştırma/'
os.makedirs(output_directory, exist_ok=True)

# Her 50 FITS dosyası için ayrı bir Excel dosyası oluştur
batch_size = 50
for i in range(0, len(fits_file_paths), batch_size):
    batch_fits_files = fits_file_paths[i:i + batch_size]

    # Excel dosyasının ismini belirle
    excel_file_name = f'batch_{i // batch_size + 1}.xlsx'
    excel_file_path = os.path.join(output_directory, excel_file_name)

    # Her FITS dosyası için döngü
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        for fits_file_path in batch_fits_files:
            # Open the FITS file
            with fits.open(fits_file_path) as hdul:
                # Access the data and header of the LINES BinTableHDU (hdul[2])
                data = hdul[1].data

                # Convert the FITS data to a pandas DataFrame
                df = pd.DataFrame(data)

                # Sadece belirtilen sütunları seç
                selected_columns = ['WlenRest', 'Flux', 'Uncertainty', 'Ion', 'DereddenedFlux', 'DereddenedFluxLo', 'DereddenedFluxHi']
                df_selected = df[selected_columns]

                # Negatif ve sıfıra eşit olan değerlere sahip satırları filtrele ve sil
                df_selected = df_selected[df_selected['Flux'] > 0.01]

                # Sayfa adını dosya isminin _id karakterinden sonrasına ayarla
                #sheet_name = os.path.basename(fits_file_path).split('_id')[1]
                sheet_name = os.path.basename(fits_file_path)[:30]

                # Verileri içeren DataFrame'i yeni bir sayfada kaydet
                df_selected.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Data has been successfully exported to {excel_file_path}")

Data has been successfully exported to /Users/nurullah/Desktop/galaxy_abudance/sigma_karşılaştırma/batch_1.xlsx


In [7]:
import pandas as pd
import os

# Excel dosyalarının bulunduğu klasörün tam yolu
excel_folder_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma4/'

# Tüm Excel dosyalarını al
excel_files = [os.path.join(excel_folder_path, file) for file in sorted(os.listdir(excel_folder_path)) if file.endswith('.xlsx')]

# Birleştirilecek verileri tutacak ana DataFrame'i oluştur
main_df = pd.DataFrame()

# Her Excel dosyası için döngü
for excel_file_path in excel_files:
    # Excel dosyasını oku
    df = pd.read_excel(excel_file_path, sheet_name=None)

    # Her sayfa için döngü
    for sheet_name, sheet_data in df.items():
        # 'WlenRest', 'Ion', 'Flux', 'Uncertainty' sütunlarını içeren sayfaları seç
        #if all(col in sheet_data.columns for col in ['WlenRest', 'Ion', 'Flux', 'Uncertainty']):
        if all(col in sheet_data.columns for col in ['WlenRest', 'Ion', 'Flux']):
            # Eğer ana DataFrame boşsa, direkt ekle
            if main_df.empty:
                #main_df = sheet_data[['WlenRest', 'Ion', 'Flux', 'Uncertainty']].copy()
                main_df = sheet_data[['WlenRest', 'Ion', 'Flux']].copy()
                #main_df.rename(columns={'Flux': f'Flux_{sheet_name}', 'Uncertainty': f'Uncertainty_{sheet_name}'}, inplace=True)
                main_df.rename(columns={'Flux': f'Flux_{sheet_name}'}, inplace=True)
            else:
                # Ana DataFrame'de 'WlenRest' ve 'Ion' sütunlarına göre birleştirme yap
                #main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'Flux', 'Uncertainty']],
                main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'Flux']],
                                        on=['WlenRest', 'Ion'],
                                        how='outer',
                                        suffixes=('', f'_{sheet_name}'))

# NaN değerleri '-' ile değiştir
main_df.fillna('-', inplace=True)

# 'WlenRest' sütununa göre küçükten büyüğe sırala
main_df.sort_values('WlenRest', inplace=True)

# Excel dosyasına ekleyeceğimiz sayfa adı
new_sheet_name = 'Flux_Tablo'

# Yeni bir Excel dosyası oluştur
output_excel_file_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma4/Fluxtablo.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    main_df.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(f"Merged, sorted data has been successfully added to the sheet {new_sheet_name} in the Excel file.")

Merged, sorted data has been successfully added to the sheet Flux_Tablo in the Excel file.


/var/folders/ry/6rcrw5vs4f33lx3wyt4f6m5w0000gn/T/ipykernel_62689/1602023791.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  main_df.fillna('-', inplace=True)


In [5]:
import pandas as pd
import os

# Excel dosyalarının bulunduğu klasörün tam yolu
excel_folder_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma4/'

# Tüm Excel dosyalarını al
excel_files = [os.path.join(excel_folder_path, file) for file in sorted(os.listdir(excel_folder_path)) if file.endswith('.xlsx')]

# Birleştirilecek verileri tutacak ana DataFrame'i oluştur
main_df = pd.DataFrame()

# Her Excel dosyası için döngü
for excel_file_path in excel_files:
    # Excel dosyasını oku
    df = pd.read_excel(excel_file_path, sheet_name=None)

    # Her sayfa için döngü
    for sheet_name, sheet_data in df.items():
        # 'WlenRest', 'Ion', 'Flux', 'Uncertainty' sütunlarını içeren sayfaları seç
        #if all(col in sheet_data.columns for col in ['WlenRest', 'Ion', 'Flux', 'Uncertainty']):
        if all(col in sheet_data.columns for col in ['WlenRest', 'Ion', 'DereddenedFlux']):
            # Eğer ana DataFrame boşsa, direkt ekle
            if main_df.empty:
                #main_df = sheet_data[['WlenRest', 'Ion', 'Flux', 'Uncertainty']].copy()
                main_df = sheet_data[['WlenRest', 'Ion', 'DereddenedFlux']].copy()
                #main_df.rename(columns={'Flux': f'Flux_{sheet_name}', 'Uncertainty': f'Uncertainty_{sheet_name}'}, inplace=True)
                main_df.rename(columns={'DereddenedFlux': f'DereddenedFlux_{sheet_name}'}, inplace=True)
            else:
                # Ana DataFrame'de 'WlenRest' ve 'Ion' sütunlarına göre birleştirme yap
                #main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'Flux', 'Uncertainty']],
                main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'DereddenedFlux']],
                                        on=['WlenRest', 'Ion'],
                                        how='outer',
                                        suffixes=('', f'_{sheet_name}'))

# NaN değerleri '-' ile değiştir
main_df.fillna('-', inplace=True)

# 'WlenRest' sütununa göre küçükten büyüğe sırala
main_df.sort_values('WlenRest', inplace=True)

# Excel dosyasına ekleyeceğimiz sayfa adı
new_sheet_name = 'DereddenedFlux_Tablo'

# Yeni bir Excel dosyası oluştur
output_excel_file_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma4/DereddenedFluxtablo.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    main_df.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(f"Merged, sorted data has been successfully added to the sheet {new_sheet_name} in the Excel file.")

Merged, sorted data has been successfully added to the sheet DereddenedFlux_Tablo in the Excel file.


/var/folders/ry/6rcrw5vs4f33lx3wyt4f6m5w0000gn/T/ipykernel_62689/842321766.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  main_df.fillna('-', inplace=True)
